In [15]:
# IMPORT MAIN LIBRARIES

import numpy as np
import imageio
import scipy as sio
import os
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt


In [16]:
# USE MLXTEND TO READ MNIST DATA

from mlxtend.data import loadlocal_mnist
import platform

if not platform.system() == 'Windows': 
  X, Y = loadlocal_mnist(
    images_path='/Users/purthasmacbookpro/Desktop/Data Structures Project/MNIST/train-images.idx3-ubyte', 
    labels_path='/Users/purthasmacbookpro/Desktop/Data Structures Project/MNIST/train-labels.idx1-ubyte'
  )
else: 
  X, Y = loadlocal_mnist(
    images_path='/Users/purthasmacbookpro/Desktop/Data Structures Project/MNIST/train-images.idx3-ubyte', 
    labels_path='/Users/purthasmacbookpro/Desktop/Data Structures Project/MNIST/train-labels.idx1-ubyte'
  )

In [17]:
# USE SKIAMGE FOR IMAGE PROCESSING SPECIFICALLY RADON 

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from skimage.transform import resize
from scipy.ndimage import zoom
from numpy import asarray
from skimage.filters import threshold_mean

X = X.astype('float32')

radonProjAngle = 4 
downSampling = 28

# create array to save all barcodes for comparisons using hamming distance
hammingSet = [] 

df = pd.DataFrame([])

# ALGORITHM: GENERATE RADON BARCODE 

for i in range(60000): 
  projAngle = 0 
  radonBarCode = np.array([])
  while projAngle < 180: 
    data = np.array([])
    thresholdProj = radon(X[i].reshape(downSampling, downSampling), theta = [projAngle])
    threshold = threshold_mean(thresholdProj)
    bin = thresholdProj >= threshold
    data = np.asanyarray(bin)
    radonBarCode = np.append(radonBarCode, data)
    projAngle = projAngle + (180 / radonProjAngle)
    radonBarCode = np.expand_dims(radonBarCode, axis = 0)
  # df.to_csv('GeneratedBarcodeDataSet.csv', index=False)
  hammingSet.append(radonBarCode)
print('Successfully created dataset for barcodes')
print('Barcodes have been generated')


Successfully created dataset for barcodes
Barcodes have been generated


In [38]:
# ALGORITHM: RADON BARCODE SEARCH 

from scipy.spatial.distance import hamming
import random
import timeit
from matplotlib import pyplot


potential = 1 
value = 0 
potential_label = 0 
count = 0 
sum = 0 

for i in range(1000): 
  x = random.randint(1, 1000)
  print('Testing Label: ' , Y[x], ' image in reference is ', x)
  potential = 1
  value = 0 
  potential_label = 0 
  sum = sum + 1
  for j in range(1000): 
    value = hamming(hammingSet[x], hammingSet[j])
    if (value < potential and x!=j):
      potential = value 
      potential_label = j 
      print('The potential label: ', Y[j], ' and the image in reference is ', j, ' and the hamming value is ', potential)
  if (Y[potential_label] == Y[x]):
    count = count + 1
    average = count/sum 
print('The average hit ratio of the dataset is ', average*100,'%')


Testing Label:  9  image in reference is  195
The potential label:  5  and the image in reference is  0  and the hamming value is  0.1875
The potential label:  9  and the image in reference is  4  and the hamming value is  0.14285714285714285
The potential label:  3  and the image in reference is  7  and the hamming value is  0.125
The potential label:  7  and the image in reference is  15  and the hamming value is  0.11607142857142858
The potential label:  9  and the image in reference is  19  and the hamming value is  0.07142857142857142
The potential label:  9  and the image in reference is  33  and the hamming value is  0.0625
The potential label:  9  and the image in reference is  43  and the hamming value is  0.044642857142857144
Testing Label:  7  image in reference is  349
The potential label:  5  and the image in reference is  0  and the hamming value is  0.22321428571428573
The potential label:  9  and the image in reference is  4  and the hamming value is  0.1785714285714285